## Question 1 (a)

Import moudles

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras import layers

2022-03-09 23:36:36.201999: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-09 23:36:36.202050: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


準備資料集

In [2]:
train_data = pd.read_csv("HW1-1.csv")
train_data.head()

,x,y
0,-0.054504,-0.223653
1,0.724703,-0.363040
2,0.269098,-0.383166
3,-0.258074,-0.154802
4,-0.311265,-0.148746


In [3]:
train_features = train_data.copy()
train_labels   = train_features.pop('y')
train_features = np.array(train_features)

建構模型

In [4]:
model = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(32),
    layers.Dense(16),
    layers.Dense(1)
])

model.compile(loss = tf.losses.MeanSquaredError(),
              optimizer = tf.optimizers.Adam())

2022-03-09 23:37:10.430999: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-09 23:37:10.431046: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-09 23:37:10.431078: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (haeter-notebook): /proc/driver/nvidia/version does not exist
2022-03-09 23:37:10.431435: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


訓練

In [5]:
model.fit(train_features, train_labels, epochs=10, verbose=0)

結果

In [6]:
test_loss = model.evaluate(x=train_features, y=train_labels)
print("Test loss: ", test_loss)

1/1 [==============================] - 0s 150ms/step - loss: 0.0301
Test loss:  0.030070578679442406


## Question 1 (b)

準備資料集

In [7]:
ORDER = 32

# Prepare data
dataframe = pd.read_csv("HW1-1.csv", index_col=None, dtype=np.float32)
dataframe.rename(columns={"x":"x_1", "y":"y"}, inplace=True)

# Calculate x to x^n, x^n-1, ..., x^1
for n in range(2, ORDER+1):
    dataframe[f"x_{n}"] = dataframe[f"x_{n-1}"] * dataframe["x_1"]

dataframe = dataframe.reindex(columns=[f"x_{n}" for n in range(1, ORDER+1)]+['y'])

training_set = dataframe.sample(frac=0.8, random_state=200)
test_set = dataframe.drop(training_set.index)

建構模型

In [8]:
model = tf.keras.Sequential([
    layers.Input(shape=(ORDER,)),
    layers.Dense(1),
])

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.MeanSquaredError()
)

訓練

In [9]:
MAX_NUMBER_OF_EPOCH=10
for i in range(MAX_NUMBER_OF_EPOCH):
    # Fit
    model.fit(training_set.drop('y', axis=1), training_set['y'], epochs=20, verbose=0)

    # Evaluate model
    test_loss = model.evaluate(x=test_set.drop('y', axis=1), y=test_set['y'])
    if test_loss < 0.1:
        break

assert test_loss < 0.1

1/1 [==============================] - 0s 24ms/step - loss: 0.0653


結果

In [10]:
test_loss = model.evaluate(x=test_set.drop('y', axis=1), y=test_set['y'])
print("Test loss: ", test_loss)

1/1 [==============================] - 0s 34ms/step - loss: 0.0653
Test loss:  0.0652788057923317


第 i 小題

In [11]:
print("i. Print out the coefficients of the equation.")
kernel, biases = model.get_layer(index=0).weights
print("Weight: ")
kernel_arr = kernel.value().numpy()
for i in range(len(kernel_arr)):
    print(f"w_{i} = {kernel_arr[i]}")
print("Bias = ", end='')
print(sum(biases.value().numpy()))

i. Print out the coefficients of the equation.
Weight: 
w_0 = [-0.04163394]
w_1 = [0.20274855]
w_2 = [-0.30061528]
w_3 = [-0.29286554]
w_4 = [-0.25009477]
w_5 = [0.01730276]
w_6 = [0.33750138]
w_7 = [0.2666694]
w_8 = [-0.24215078]
w_9 = [0.02877681]
w_10 = [-0.33763015]
w_11 = [-0.3468432]
w_12 = [0.31907126]
w_13 = [-0.18169421]
w_14 = [-0.07163016]
w_15 = [0.3385871]
w_16 = [0.4587047]
w_17 = [-0.30409378]
w_18 = [-0.03148082]
w_19 = [0.15779418]
w_20 = [0.48006257]
w_21 = [-0.33570647]
w_22 = [0.01507698]
w_23 = [-0.34737223]
w_24 = [0.13582017]
w_25 = [-0.31551775]
w_26 = [-0.11451023]
w_27 = [-0.01853955]
w_28 = [0.39415994]
w_29 = [-0.12617722]
w_30 = [0.0638641]
w_31 = [0.21645708]
Bias = -0.05846880003809929


第 ii 小題

In [12]:
print("ii. Calculate the following gradients:")

def diff(x:tf.constant, weight: tf.Variable):
    assert weight.shape.ndims == 2
    assert weight.shape[-1] == 1

    x_power = np.ndarray(shape=weight.shape, dtype=np.float32)
    x_power[0] = 1

    for i in range(1, len(x_power)):
        x_power[i] = x_power[i-1] * x.numpy()

    x_power = tf.Variable(x_power)

    values = tf.matmul(weight, x_power, transpose_b=True)
    return tf.reduce_sum(values)

diff_value = diff(tf.Variable(2.0, dtype=tf.float32), kernel)
print(f"f'(2.0)= {diff_value}")

diff_value = diff(tf.Variable(0.5, dtype=tf.float32), kernel)
print(f"f'(0.5)= {diff_value}")

diff_value = diff(tf.Variable(-0.8, dtype=tf.float32), kernel)
print(f"f'(-0.8)= {diff_value}")

ii. Calculate the following gradients:
f'(2.0)= -970487232.0
f'(0.5)= -0.4519183337688446
f'(-0.8)= -0.12543338537216187
